# NBL-V3 Player-Only Training
## Specialized model for player and referee detection

**Why separate player model?**
- Players are the PRIMARY objects (most important)
- Need high accuracy for tracking and team assignment
- Can optimize for congestion scenarios
- Different confidence thresholds for player vs referee

# Install Libraries

In [ ]:
!pip install ultralytics
!pip install roboflow

In [ ]:
!pip freeze | grep roboflow

# Imports

In [ ]:
from roboflow import Roboflow
import shutil
import yaml

# Get Dataset

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="ZzD21wz5oTPdE0fhb04C")
project = rf.workspace("tomatoes-iicln").project("nbl")
version = project.version(3)
dataset = version.download("yolov11")

In [ ]:
shutil.move("NBL-3/train",
"NBL-3/NBL-3/train",
)
shutil.move("NBL-3/valid",
"NBL-3/NBL-3/valid",
)

# Filter Dataset for Players Only

We need to modify data.yaml to only include 'player' and 'referee' classes

In [ ]:
# Read original data.yaml
with open('/content/NBL-3/data.yaml', 'r') as f:
    data = yaml.safe_load(f)

print("Original classes:", data['names'])

# Create player-only data.yaml
player_data = {
    'train': data['train'],
    'val': data['val'],
    'test': data['test'],
    'nc': 2,
    'names': ['player', 'referee']
}

# Save player-only config
with open('/content/NBL-3/data_players_only.yaml', 'w') as f:
    yaml.dump(player_data, f)

print("\nCreated data_players_only.yaml")
print("Training will focus on player and referee detection")

# Training - Optimized for Player Detection

In [ ]:
from ultralytics import YOLO

# Use YOLOv11s for better accuracy on players
model = YOLO('yolo11s.pt')

results = model.train(
    data='/content/NBL-3/data_players_only.yaml',
    epochs=100,
    batch=16,
    imgsz=1080,  # High resolution for player details
    patience=50,
    device=0,
    workers=8,
    project='nbl_v3_players_only',
    name='train',
    exist_ok=False,
    pretrained=True,
    optimizer='auto',
    verbose=True,
    seed=0,
    deterministic=True,
    val=True,
    plots=True,
    # Player-specific optimizations
    mosaic=1.0,      # Good for congestion
    mixup=0.1,       # Helps with overlapping players
    degrees=5.0,     # Rotation for different camera angles
)

# Download Model

Model will be saved as: `nbl_v3_players_only/train/weights/best.pt`

Use this in your config:
```python
PLAYER_DETECTOR_PATH = 'models/nbl_v3_players.pt'
```